# Word2Vec with Gensim

In [3]:
from gensim.models import KeyedVectors
from os.path import join as pathjoin;

## Configs

In [4]:
base_path = 'C:/Users/landrade/Dropbox/python/dbdc3/'
nilc300 = pathjoin(base_path,'Nilc_embeddings/skip_s300.txt')

In [5]:
# carregar embeddings pre-treinados (NILC ou que vc mesmo salvou usando o procedimento a seguir)
w2v = KeyedVectors.load_word2vec_format(nilc300)

In [16]:
from scipy.sparse import issparse
print(issparse(w2v['leonardo']))

False


In [1]:
        


#--------------------------------------------------------------------
# treinar embeddings a partir do próprio texto e salvar em um arquivo de embeddings

# cria vetor TFIDF e estruturas auxiliares (x_tfidf, vocab, dic e fanalyzer) 
# que serão usadas na criação dos embeddings ponderados depois, 
# a partir de uma coluna de texto "textcol" do dataframe
text_column = df.loc[:,textcol].str.lower()                          
text_column = text_column.values
vect = TfidfVectorizer()
x_tfidf = vect.fit_transform(df[textcol])
vocab = vect.get_feature_names()
dic = vect.vocabulary_
fanalyzer = vect.build_analyzer() 

# transforma cada documento da coluna de textos em uma lista de tokens 
tokenList = []
for item in text_column:
    tokens = fanalyzer(item)
    tokenList.append(tokens)

# treina embeddings a partir do conjunto de listas de tokens criado
model = Word2Vec(tokenList, size=300, window=1, min_count=1, sg=1, negative=10, hs=0, iter=5, workers=1, seed=10)            

# salva modelo de embeddings criado
model.wv.save(nomedoarquivo+".kv")

#---------------------------------------------------------------

# usando um modelo de embeddings para criar o vetor X de (300?) features de um documento

# cria features X (de treinamento ou teste) a partir de uma coluna de texto
# depois basta usar x_w2v como atributos de aprendizado em um classificador qualquer
x_w2v = []
dc = 0 # contador de documentos na coluna de texto
for doc in text_col:
	tokens = fanalyzer(doc)   # transforma texto em lista de tokens
	features = x_tfidf[dc,]   # contagens tfidf originais
	x_w2v.append(getWeightedDocumentVector(w2v, tokens, features, dic))
	dc = dc + 1

# dado um documento (na forma de uma lista de tokens), retorna o embedding ponderado por TFIDF
def getWeightedDocumentVector(w2v, tokens, features, dic):
    vectors = []
    tokens = [word for word in tokens if (word in w2v.vocab) ]
    for word in tokens:
        word_index = dic[word]
        word_vector = w2v[word]  # numpy vector of a given word
        word_tfidf = features[0,word_index]
        vectors.append(word_vector * word_tfidf)
    v = np.zeros(w2v.vector_size)
    if len(vectors)==0:
        return v
    else:
        for vector in vectors:
            v = v + vector
        return (v / len(vectors))

NameError: name 'KeyedVectors' is not defined